In [58]:
import sys
import numpy as np
import math

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.extensions import Initialize

In [59]:
HORSE    = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTTTGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCAAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTTCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
THEILON  = "GTGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTTTGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCAAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTTCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
DUCK     = "ATGGTGCTGTCTGCGGCTGACAAGACCAACGTCAAGGGTGTCTTCTCCAAAATCGGTGGCCATGCTGAGGAGTATGGCGCCGAGACCCTGGAGAGGATGTTCATCGCCTACCCCCAGACCAAGACCTACTTCCCCCACTTTGACCTGCAGCACGGCTCTGCTCAGATCAAGGCCCATGGCAAGAAGGTGGCGGCTGCCCTAGTTGAAGCTGTCAACCACATCGATGACATTGCGGGTGCTCTCTCCAAGCTCAGTGACCTCCACGCCCAAAAGCTCCGTGTGGACCCTGTCAACTTCAAATTCCTGGGCCACTGCTTCCTGGTGGTGGTTGCCATCCACCACCCCGCTGCCCTGACCCCAGAGGTCCACGCTTCCCTGGACAAGTTCATGTGCGCCGTGGGTGCTGTGCTGACTGCCAAGTACCGTTAGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"
GOAT     = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCAGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCACCCCAGTGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

In [46]:
def encode_bitstring(bitstring, qr, cr, inverse=False):
  
    n = math.ceil(math.log2(len(bitstring))) + 2                 #nímero de qubits, len(bitstrings)=512
    assert n > 2, 
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #desired vector inicializado con 0s


    
    qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inicialización
    inverse_qc_init = QuantumCircuit(n) # Creación de compuertas circuitales para inversión de la inicialización



    amplitude = np.sqrt(1.0/2**(n-2))             #Se define la amplitud de cada segmento
    print(amplitude)                              #Con este print verificamos el valor de la amplitud
    pr=0
    for i, b in enumerate(bitstring):
        pos = i * 4                               #A por defecto estara en 0
            
        if b == "1" or b == "C":                  #Se verifica el nucleotido C
            pos += 1
        elif b == "2" or b == "G":                 #Se verifica el nucleotido G
            pos += 2
        elif b == "3" or b == "T":                 #Se verifica el nucleotido T
            pos += 3
        
        desired_vector[pos] = amplitude            #En la posición que este parado asigna el valor de la amplitud
        pr=amplitude**2+pr               
  
 #         Cambio requerido en el código para corregir error presentado   #
    
    print(pr)                                       #Con este print verificamos que se cumpla que la suma de amplitudes es 1
   
    qc_init = Initialize(desired_vector)
    
      
    
    if not inverse:


        qc.append(qc_init, qr) 
    
        qc.barrier(qr)
        
    else:


        inverse_qc_init = qc_init.gates_to_uncompute() 
        qc.append(inverse_qc_init, qr)  #
        
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    print()
    return qc

In [47]:
n = math.ceil(math.log2(len(HORSE))) + 2                 #nímero de qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_horse     = encode_bitstring(HORSE, qr, cr)                 #Definimos la cadena de HORSE
qc_theilon     = encode_bitstring(THEILON, qr, cr)             #Definimos la cadena de THEILON
qc_duck = encode_bitstring(DUCK, qr, cr)                       #Definimos la cadena de DUCK
qc_goat = encode_bitstring(GOAT, qr, cr)                       #Definimos la cadena de GOAT


circs = {"HORSE": qc_horse, "DUCK": qc_duck, "GOAT": qc_goat, "THEILON": qc_theilon}

In [48]:
inverse_qc_horse     = encode_bitstring(HORSE,     qr, cr, inverse=True)        #INVERTIMOS la cadena de HORSE
inverse_qc_theilon     = encode_bitstring(THEILON,     qr, cr, inverse=True)    #INVERTIMOS la cadena de THEILON
inverse_qc_duck = encode_bitstring(DUCK, qr, cr, inverse=True)                  #INVERTIMOS la cadena de DUCK
inverse_qc_goat = encode_bitstring(GOAT, qr, cr, inverse=True)                  #INVERTIMOS la cadena de GOAT


inverse_circs = {"HORSE": inverse_qc_horse, "DUCK": inverse_qc_duck, "GOAT": inverse_qc_goat, "THEILON": inverse_qc_theilon}

In [49]:
from qiskit import IBMQ, BasicAer

key = "HORSE"                                                 #Key sera la cadena respecto a la cual compararemos

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 8192

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:                               #Comparamos cada cadena respecto a HORSE
 
        
    combined_circs[other_key] = circs[key].compose(inverse_circs[other_key])  
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)                   #Mostramos la similitud respecto a cada cadena
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)

Similarity score of HORSE and HORSE is 1.0
Similarity score of HORSE and DUCK is 0.60595703125
Similarity score of HORSE and GOAT is 0.801025390625
Similarity score of HORSE and THEILON is 0.9971923828125
[ANSWER] HORSE is most similar to HORSE


In [54]:
print(combined_circs[key])

           »
 q57311_0: »
           »
 q57311_1: »
           »
 q57311_2: »
           »
 q57311_3: »
           »
 q57311_4: »
           »
 q57311_5: »
           »
 q57311_6: »
           »
 q57311_7: »
           »
 q57311_8: »
           »
 q57311_9: »
           »
q57311_10: »
           »
   c15: 11/»
           »
«           ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [57]:
print(circs[key])

           »
 q57311_0: »
           »
 q57311_1: »
           »
 q57311_2: »
           »
 q57311_3: »
           »
 q57311_4: »
           »
 q57311_5: »
           »
 q57311_6: »
           »
 q57311_7: »
           »
 q57311_8: »
           »
 q57311_9: »
           »
q57311_10: »
           »
   c15: 11/»
           »
«           ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────